# Get libraries and repo names with Python and R scripts

```SQL
SELECT DISTINCT module, repo_name, is_python
FROM (
  SELECT file_id, repo_name, path, line, ENDS_WITH(path, '.py') AS is_python, IF(
    ENDS_WITH(path, '.py'), IF(
      REGEXP_CONTAINS(line, r'^import '),
        REGEXP_EXTRACT_ALL(line, r'(?:^import|,) ([a-zA-Z0-9\_]+)'),
        REGEXP_EXTRACT_ALL(line, r'^from ([a-zA-Z0-9\_]+)')
      ),
      REGEXP_EXTRACT_ALL(line, r'library\s*\(([a-zA-Z0-9\_]+)\s*\)')
    ) AS modules
  FROM (
    SELECT
          ct.id AS file_id, repo_name, path,
          # Add a space after each line.
          # It is required to ensure correct line numbering.
          SPLIT(REPLACE(content, "\n", " \n"), "\n") AS lines
    FROM `bigquery-public-data.github_repos.sample_files` AS fl
    JOIN `bigquery-public-data.github_repos.sample_contents` AS ct ON fl.id=ct.id), UNNEST(lines) as line
  WHERE
    (ENDS_WITH(path, '.py') AND (REGEXP_CONTAINS(line, "^import ") OR REGEXP_CONTAINS(line, "^from .* import "))) OR 
    (ENDS_WITH(path, '.r') AND REGEXP_CONTAINS(line, r"library\s*\("))
  ORDER BY
    repo_name, file_id
), UNNEST(modules) as module
ORDER BY
  repo_name
```

Sample 2:


```SQL
SELECT DISTINCT module, repo_name, is_python, language.name
FROM (
  SELECT file_id, repo_name, path, line, language, ENDS_WITH(path, '.py') AS is_python, IF(
    ENDS_WITH(path, '.py'), IF(
      REGEXP_CONTAINS(line, r'^import '),
        REGEXP_EXTRACT_ALL(line, r'(?:^import|,) ([a-zA-Z0-9\_]+)'),
        REGEXP_EXTRACT_ALL(line, r'^from ([a-zA-Z0-9\_]+)')
      ),
      REGEXP_EXTRACT_ALL(line, r'library\s*\(([a-zA-Z0-9\_]+)\s*\)')
    ) AS modules
  FROM (
    SELECT
          ct.id AS file_id, fl.repo_name, path, ln.language,
          # Add a space after each line.
          # It is required to ensure correct line numbering.
          SPLIT(REPLACE(content, "\n", " \n"), "\n") AS lines
    FROM `bigquery-public-data.github_repos.sample_files` AS fl
    JOIN `bigquery-public-data.github_repos.languages` AS ln ON fl.repo_name=ln.repo_name
    JOIN `bigquery-public-data.github_repos.sample_contents` AS ct ON fl.id=ct.id), UNNEST(lines) as line
  WHERE
    (ENDS_WITH(path, '.py') AND (REGEXP_CONTAINS(line, "^import ") OR REGEXP_CONTAINS(line, "^from .* import "))) OR 
    (ENDS_WITH(path, '.r') AND REGEXP_CONTAINS(line, r"library\s*\("))
  ORDER BY
    repo_name, file_id
), UNNEST(modules) as module, UNNEST(language) as language
ORDER BY
  repo_name
```

https://stackoverflow.com/questions/50499049/bigquery-export-query-results-to-local-file-google-storage